In [5]:
from datasets import load_dataset
from rich.console import Console
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from evaluation.code_execution import print_code_snippet, Language
console = Console()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
cc_dataset = load_dataset("deepmind/code_contests")["train"]

In [30]:
from evaluation.code_execution import evaluate_python_code_exec


console = Console()
n_ran = 0
for i, problem in enumerate(cc_dataset):
    description = problem["description"]
    # display(Markdown(f"### Problem {i}: {problem['name']}\n{description}"))
    # print(i)

    for j, (lang_txt, solution_code) in enumerate(
        zip(problem["solutions"]["language"], problem["solutions"]["solution"])
    ):
        lang_enum = Language(lang_txt)
        if lang_enum.value != Language.PYTHON3.value:
            continue
        # print_code_snippet(solution_code, console)
        for test_input in problem["public_tests"]['input']:
            err, res = evaluate_python_code_exec(solution_code, test_input)
            print(err, res)
        n_ran += 1
        if n_ran > 5:
            break
    
    if n_ran > 5:
        break


name 'deque' is not defined None


Traceback (most recent call last):
  File "/home/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 268, in evaluate_python_code_exec
    exec(code_to_run, exec_globals, local_vars)
  File "<string>", line 57, in <module>
  File "<string>", line 6, in solve
NameError: name 'deque' is not defined


name 'deque' is not defined None


Traceback (most recent call last):
  File "/home/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 268, in evaluate_python_code_exec
    exec(code_to_run, exec_globals, local_vars)
  File "<string>", line 57, in <module>
  File "<string>", line 6, in solve
NameError: name 'deque' is not defined


name 'deque' is not defined None


Traceback (most recent call last):
  File "/home/brian/smolmodels/notebooks/../evaluation/code_execution.py", line 268, in evaluate_python_code_exec
    exec(code_to_run, exec_globals, local_vars)
  File "<string>", line 57, in <module>
  File "<string>", line 6, in solve
NameError: name 'deque' is not defined


None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
